In [1]:
import torch

a = torch.rand(5,6,7,8)

In [6]:
(a.shape[0], + 1,)*(a.ndim-1)

(5, 1, 5, 1, 5, 1)

In [5]:
a.ndim-1

3

In [7]:
aaa  = torch.rand((4,1,1,1,1))

In [9]:
a

tensor([[[[0.8734, 0.7546, 0.4107,  ..., 0.7972, 0.9881, 0.7353],
          [0.7520, 0.4873, 0.3946,  ..., 0.3766, 0.7400, 0.4296],
          [0.3668, 0.4034, 0.2728,  ..., 0.9406, 0.3254, 0.5981],
          ...,
          [0.5145, 0.1129, 0.0303,  ..., 0.3213, 0.2042, 0.1613],
          [0.8166, 0.6152, 0.0367,  ..., 0.9805, 0.3592, 0.6301],
          [0.4300, 0.4130, 0.5088,  ..., 0.7145, 0.7846, 0.8763]],

         [[0.1773, 0.2812, 0.4533,  ..., 0.1468, 0.6121, 0.3595],
          [0.4660, 0.2312, 0.8779,  ..., 0.9052, 0.0113, 0.7876],
          [0.9612, 0.0249, 0.1306,  ..., 0.3103, 0.2675, 0.4631],
          ...,
          [0.0020, 0.6623, 0.5289,  ..., 0.6446, 0.1159, 0.6806],
          [0.5992, 0.1158, 0.1142,  ..., 0.5587, 0.5736, 0.9890],
          [0.5014, 0.3853, 0.3451,  ..., 0.2594, 0.1131, 0.5614]],

         [[0.8982, 0.4145, 0.0793,  ..., 0.3067, 0.0753, 0.0052],
          [0.7063, 0.6566, 0.4647,  ..., 0.7077, 0.6467, 0.9491],
          [0.1051, 0.7757, 0.3995,  ..., 0

In [13]:
torch.floor_(a)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from functools import partial

class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample"""
    def __init__(self, drop_prob: float = 0.):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        return x.div(keep_prob) * random_tensor


class Mlp(nn.Module):
    """Feed‑forward layer with GELU activation"""
    def __init__(self, in_features, hidden_features=None, out_features=None, dropout=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return self.dropout(x)


class Attention(nn.Module):
    """Multi‑head Self‑Attention"""
    def __init__(self, dim, num_heads=8, qkv_bias=False, attn_dropout=0., proj_dropout=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_dropout)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_dropout)

    def forward(self, x: torch.tensor, mask: torch.tensor = None):
        B, N, C = x.shape
        qkv = self.qkv(x)  # (B, N, 3*C)
        qkv = qkv.reshape(B, N, 3, self.num_heads, C // self.num_heads)
        q, k, v = qkv.permute(2, 0, 3, 1, 4)  # each (B, heads, N, head_dim)

        attn = (q @ k.transpose(-2, -1)) * self.scale

        if mask is not None:
            attn = attn + mask

        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        return self.proj_drop(x)


class Block(nn.Module):
    """Transformer Encoder Block with Pre‑Norm, DropPath"""
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=True,
                 drop=0., attn_drop=0., drop_path=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = Attention(dim, num_heads=num_heads,
                              qkv_bias=qkv_bias,
                              attn_dropout=attn_drop,
                              proj_dropout=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = nn.LayerNorm(dim)
        hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(dim, hidden_dim, dropout=drop)

    def forward(self, x):
        # Pre‑Norm
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class ImageEncoder(nn.Module):
    """
    ViT‑style Image Encoder used in BLIP1
    (patch embedding + positional embedding + transformer blocks + final norm)
    """
    def __init__(
        self,
        image_size: int = 224,
        patch_size: int = 16,
        embed_dim: int = 768,
        depth: int = 12,
        num_heads: int = 12,
        mlp_ratio: float = 4.0,
        qkv_bias: bool = True,
        drop_rate: float = 0.0,
        attn_drop_rate: float = 0.0,
        drop_path_rate: float = 0.1,
    ):
        super().__init__()
        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = (image_size // patch_size) ** 2

        # patch embed
        self.patch_embed = nn.Conv2d(3, embed_dim,
                                     kernel_size=patch_size,
                                     stride=patch_size)
        # class token + positional embedding
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        # stochastic depth decay rule
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]
        # transformer blocks
        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias, drop=drop_rate,
                attn_drop=attn_drop_rate, drop_path=dpr[i]
            )
            for i in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)

        # initialize
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            nn.init.zeros_(m.bias)
            nn.init.ones_(m.weight)

    def forward(self, x: torch.Tensor):
        """
        x: (B, 3, H, W)
        returns: (B, num_patches+1, embed_dim)  — đầu tiên là CLS token
        """
        B = x.shape[0]
        # patch embedding
        x = self.patch_embed(x)                   # (B, C, H/ps, W/ps)
        x = x.flatten(2).transpose(1, 2)          # (B, N, C)

        # prepend cls token
        cls_tokens = self.cls_token.expand(B, -1, -1)  # (B, 1, C)
        x = torch.cat((cls_tokens, x), dim=1)          # (B, N+1, C)
        x = x + self.pos_embed                         # add pos embed
        x = self.pos_drop(x)

        # transformer encoder
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        return x


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from functools import partial

class DropPath(nn.Module):
    """Stochastic Depth per sample"""
    def __init__(self, drop_prob: float = 0.):
        super().__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        return x.div(keep_prob) * random_tensor

class Mlp(nn.Module):
    """Feed-forward layer with GELU"""
    def __init__(self, in_features, hidden_features=None, out_features=None, dropout=0.):
        super().__init__()
        hidden = hidden_features or in_features
        out = out_features or in_features
        self.fc1 = nn.Linear(in_features, hidden)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden, out)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        return self.drop(x)

class SelfAttention(nn.Module):
    """Multi-head self-attention"""
    def __init__(self, dim, num_heads=8, qkv_bias=True, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads)
        q, k, v = qkv.permute(2, 0, 3, 1, 4)
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        return self.proj_drop(x)

class CrossAttention(nn.Module):
    """Cross-attention: text queries, image keys/values"""
    def __init__(self, dim, num_heads=8, qkv_bias=True, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.q = nn.Linear(dim, dim, bias=qkv_bias)
        self.kv = nn.Linear(dim, dim * 2, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x_text, x_image):
        B, Nt, C = x_text.shape
        _, Ni, _ = x_image.shape

        q = self.q(x_text).reshape(B, Nt, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        kv = self.kv(x_image).reshape(B, Ni, 2, self.num_heads, C // self.num_heads)
        k, v = kv.permute(2, 0, 3, 1, 4)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, Nt, C)
        x = self.proj(x)
        return self.proj_drop(x)

class TextBlock(nn.Module):
    """Transformer block for text (pre-norm)"""
    def __init__(self, dim, num_heads, mlp_ratio=4., drop=0., attn_drop=0., drop_path=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = SelfAttention(dim, num_heads, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = Mlp(dim, int(dim * mlp_ratio), dropout=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

class TextEncoder(nn.Module):
    """BERT-like unimodal text encoder"""
    def __init__(self,
                 vocab_size: int,
                 max_len: int = 512,
                 embed_dim: int = 768,
                 depth: int = 12,
                 num_heads: int = 12,
                 mlp_ratio: float = 4.,
                 drop_rate: float = 0.1,
                 attn_drop_rate: float = 0.1,
                 drop_path_rate: float = 0.0):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, embed_dim)
        self.pos_embed = nn.Parameter(torch.zeros(1, max_len, embed_dim))
        self.drop = nn.Dropout(drop_rate)

        dpr = torch.linspace(0, drop_path_rate, depth)
        self.blocks = nn.ModuleList([
            TextBlock(embed_dim, num_heads, mlp_ratio,
                      drop_rate, attn_drop_rate, dpr[i])
            for i in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)
        self.max_len = max_len

        nn.init.trunc_normal_(self.pos_embed, std=0.02)

    def forward(self, token_ids):
        # token_ids: (B, T)
        B, T = token_ids.shape
        pos = self.pos_embed[:, :T]
        x = self.token_embed(token_ids) + pos
        x = self.drop(x)
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        return x  # (B, T, dim)

class ImageGroundedTextEncoder(nn.Module):
    """Inject cross-attention layers for ITM"""
    def __init__(self,
                 base_encoder: TextEncoder,
                 cross_interval: int = 1):
        super().__init__()
        self.base = base_encoder
        self.cross_idx = set(range(0, len(self.base.blocks), cross_interval))
        # make cross-attention modules
        self.cross_layers = nn.ModuleDict({
            str(i): CrossAttention(self.base.token_embed.embedding_dim,
                                   self.base.blocks[0].attn.num_heads)
            for i in self.cross_idx
        })
        self.itm_head = nn.Linear(self.base.token_embed.embedding_dim, 1)

    def forward(self, token_ids, image_feats):
        # token_ids: (B, T), image_feats: (B, N_img, dim)
        x = self.base.token_embed(token_ids) + self.base.pos_embed[:, :token_ids.size(1)]
        x = self.base.drop(x)
        for i, blk in enumerate(self.base.blocks):
            x = blk(x)
            if i in self.cross_idx:
                x = x + self.cross_layers[str(i)](x, image_feats)
        x = self.base.norm(x)
        # ITM: use first token ([Encode])
        cls_feat = x[:, 0]
        itm_logit = self.itm_head(cls_feat).squeeze(-1)
        return x, itm_logit

# Example instantiation:
# text_encoder = TextEncoder(vocab_size=30522)
# img_encoder = ImageEncoder()
# ig_text_encoder = ImageGroundedTextEncoder(text_encoder)


In [2]:
import torch
cls_token = torch.nn.Parameter(torch.zeros(1,1,768))


In [3]:
cls_token

Parameter containing:
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [17]:
b= cls_token.expand(8,-1,-1)

In [18]:
x = torch.rand(8,16,768)

In [21]:
a.shape

torch.Size([8, 17, 768])